# LangGraph를 사용한 고객 지원 챗봇을 구축

- 웹 검색을 통해 일반적인 질문에 답변 
- 대화 상태를 유지하여 연속적인 대화  
- 복잡한 질문을 사람이 검토하도록 라우팅  
- 사용자 지정 상태(Custom State)를 활용하여 챗봇의 동작 제어  
- 대화 흐름을 되돌리고(Rewind), 다른 대화 경로 탐색 

In [ ]:
# LangSmith 추적 설정 활성화

먼저 모델을 직접 사용해 봅니다. `ChatModel`은 LangChain의 **"Runnable"** 인스턴스이며, 이는 표준화된 인터페이스를 통해 상호작용할 수 있음을 의미합니다.  

모델을 간단하게 호출하려면 `.invoke` 메서드에 **메시지 목록**을 전달하면 됩니다.

모델 자체는 **상태(state)** 라는 개념을 가지고 있지 않습니다. 예를 들어, 후속 질문을 하면:

<br>
이제 좋은 응답을 받는 것을 확인할 수 있습니다!  

이것이 챗봇이 **대화형 상호작용**을 할 수 있는 기본 아이디어입니다.  

### **Part 1: 기본 챗봇 만들기¶**  

In [ ]:
# LangGraph에서 상태 그래프를 구성하기 위한 기본 클래스 및 상수 임포트
# LangGraph에서 메시지를 상태에 추가하기 위한 헬퍼 함수
# 상태(State)를 정의하는 클래스
# 이 상태는 메시지 목록(messages)을 포함하며,
# add_messages는 메시지를 자동으로 병합하거나 관리하는 역할을 함
class State(TypedDict):
# 상태 타입을 기반으로 그래프 빌더 생성
# 노드 함수 정의
# 입력 상태에서 messages를 꺼내서 LLM에게 전달하고, 
# 생성된 응답 메시지를 다시 messages에 추가하여 반환
def chatbot(state: State):
# "chatbot"이라는 이름의 노드를 그래프에 추가

- 챗봇 노드 함수는 **현재 상태(State)를 입력**으로 받고, **업데이트된 메시지 리스트**를 `"messages"` 키에 담아 **딕셔너리 형태로 반환**합니다.  
- 이 패턴은 **모든 LangGraph 노드 함수의 기본 구조**입니다.  

**다음 단계:**  
- **엔트리 포인트(Entry Point)** 를 추가  
- **엔트리 포인트**는 **그래프 실행 시 시작 지점을 정의**하며, **매번 그래프를 실행할 때 어디서부터 작업을 시작할지 지정하는 역할**을 합니다.
- 마찬가지로, **종료 지점(Finish Point)** 을 설정합니다. 이는 그래프에게 **"이 노드가 실행될 때마다, 여기서 작업을 종료할 수 있다."** 라고 지시하는 역할을 합니다.
- 마지막으로, 그래프를 실행할 수 있도록 설정해야 합니다. 이를 위해 **`compile()`** 메서드를 그래프 빌더에서 호출합니다.   

### **챗봇 실행**  

대화 루프에서 언제든지 **"quit"**, **"exit"**, 또는 **"q"** 를 입력하면 종료할 수 있습니다. 

In [ ]:
def stream_graph_updates(user_input: str):
# 무한 루프 (사용자가 "quit", "exit", "q"를 입력하면 종료)

LangGraph를 사용하여 첫 번째 챗봇을 구축 했습니다.  하지만 현재는 외부 정보도 가져오지 못하고, 사용자의 이름도 기억하지 못합니다.

다음 단계에서는 웹 검색 도구를 추가하여 챗봇의 지식을 확장하고, 더 강력한 기능을 갖추도록 만들겠습니다. 

## **Part 2: 도구를 활용한 챗봇 강화**   
이제 챗봇이 **웹에서 관련 정보를 찾아 더 나은 답변을 제공할 수 있도록 개선**합니다. 

---
먼저 **Tavily 검색 엔진**을 사용하기 위해 필요한 패키지를 설치하고 **`TAVILY_API_KEY`** 를 설정 합니다.  

In [ ]:
# %%capture --no-stderr
#%pip install -U langchain-tavily

In [ ]:
# Tavily 검색 툴을 임포트
# Tavily는 웹 검색을 통해 정보를 가져오는 도구로, LangChain에서 도구(tool)로 활용 가능
# TavilySearch 툴 인스턴스를 생성 (최대 결과 2개로 제한)
# 사용할 툴들을 리스트로 구성 (여러 개의 도구가 필요한 경우를 대비해 리스트 형태로 작성)
# Tavily 검색 도구를 직접 호출하여 "LangGraph에서 node가 뭐야?"라는 질문에 대한 웹 검색 결과를 가져옴

In [ ]:
# pip install -U "langchain[openai]"

### 도구(Tools)를 실행하는 함수 생성하기
이제, 도구가 호출되는 경우 이를 실행하는 함수를 만들어봅시다.  
이를 위해 `llm_with_tools`라는 새 노드를 만들고, 여기에 도구들을 추가합니다.  
이 노드는 상태(state)에서 가장 최근 메시지를 확인하고, 그 메시지에 tool_calls가 포함되어 있는 경우 해당 도구들을 호출합니다.

이 기능은 Anthropic, OpenAI, Google Gemini 및 여러 다른 LLM 제공업체에서 지원하는 tool calling 기능을 기반으로 동작합니다.  

In [ ]:
# LangGraph에서 상태 그래프 구성 관련 기능과 상수 가져오기
# LangGraph의 메시지 병합 기능: 메시지를 상태에 병합하거나 정리하는 헬퍼 함수
# 상태(State) 클래스 정의
# 'messages'는 Annotated를 사용해 LangGraph의 메시지 병합 기능(add_messages)을 적용
class State(TypedDict):
# 상태(State)를 기반으로 하는 그래프 빌더 생성
# 사용할 도구들을 LLM에 명시적으로 바인딩 (Tool Calling 지원)
# 이를 통해 LLM이 명시된 도구만 호출할 수 있도록 제한
# 챗봇 노드 함수 정의
# 현재 상태의 messages를 LLM에 전달하여 응답을 생성하고,
# 그 응답 메시지를 다시 상태에 포함되도록 반환
def chatbot(state: State):
# 'chatbot'이라는 이름의 노드를 그래프에 추가
# LangGraph 내부에서 이 노드는 하나의 처리 단위로 동작함

In [ ]:
# LangChain에서 tool 호출 결과를 감싸기 위한 메시지 타입
# 도구 실행 노드 정의
class BasicToolNode:
    def __init__(self, tools: list) -> None:
        # 도구 이름을 키로 하여 도구 인스턴스를 딕셔너리로 저장
        # 예: {'SearchTool': SearchTool 객체}
    def __call__(self, inputs: dict):
        # 입력에서 messages 리스트를 꺼냄 (없으면 빈 리스트)
            # 가장 마지막 메시지를 가져옴 (보통 AIMessage)
        # AIMessage에 포함된 tool_calls 순회
            # 도구 이름으로 매칭된 도구를 찾아 호출
            # 도구 호출 결과를 ToolMessage로 감싸고 outputs에 추가
        # 처리된 메시지 목록을 반환 (상태 업데이트 용도)
# 위에서 정의한 도구 실행 노드를 그래프에 추가
# "tools"라는 이름의 노드로 추가되어 이후 그래프 실행 시 이 노드를 호출함

**조건부 엣지(Conditional Edges)의 동작 방식**  

- 조건부 엣지는 특정 노드에서 시작됩니다. 즉, 'chatbot' 노드가 실행될 때마다, 다음과 같은 경로를 결정합니다.

1️) 만약 챗봇이 도구(`tool_calls`)를 호출하면 'tools' 노드로 이동   
2️) 만약 챗봇이 직접 응답하면 `END`로 이동하여 실행 종료


**`tools_condition`**  

- **`tool_calls`가 없는 경우 `END` 문자열을 반환하여 실행을 종료** 합니다.  

In [ ]:
def route_tools(state: State):
    # 상태가 리스트 타입일 경우 마지막 메시지를 가져옴
    # 상태 딕셔너리에서 'messages' 키를 통해 메시지 리스트를 가져와 마지막 메시지를 선택
        # 메시지가 없을 경우 예외 발생
    # 마지막 메시지가 tool_calls 속성을 가지고 있고, 하나 이상 도구 호출이 있을 경우
# 조건부 엣지를 추가하여, chatbot 노드 이후 흐름을 동적으로 제어
# chatbot이 도구 호출을 요청하면 tools 노드로, 아니면 END로 흐름이 결정됨
# 도구를 호출한 이후에는 다시 chatbot 노드로 돌아가서 다음 행동 결정
# 그래프의 시작점에서 chatbot 노드로 진입
# 모든 노드와 엣지가 정의된 후 그래프를 컴파일 (실행 준비 완료)

In [ ]:
    # This requires some extra dependencies and is optional

이제 챗봇에게 **훈련 데이터에 없는 질문도 할 수 있습니다.** 

In [ ]:
def stream_graph_updates(user_input: str):

이제 LangGraph에서 검색 엔진을 활용할 수 있는 대화형 에이전트(Conversational Agent)를 만들었습니다

--- 

## **Part 3: 챗봇에 메모리 기능 추가¶**  

현재 챗봇은 **사용자 질문에 도구를 활용해 답변할 수 있지만, 이전 대화의 맥락을 기억하지 못합니다.**  
이 때문에 **일관된 멀티턴(Multi-turn) 대화를 진행하는 데 한계가 있습니다.**  

LangGraph는 **"지속적 체크포인트(Persistent Checkpointing)"** 기능을 통해 이 문제를 해결합니다.  

그래프를 컴파일할 때 checkpointing을 활성화하고 그래프를 호출할 때 `thread_id`를 제공하면, LangGraph가 자동으로 상태(state)를 저장하고, 다음 실행 시 이전 상태를 복원합니다.  

즉, **동일한 `thread_id`** 를 사용하여 그래프를 호출하면, 이전 대화 상태를 불러와서 이어서 대화할 수 있습니다!

우리는 현재 **메모리를 활용하는(in-memory) 체크포인터**를 사용하고 있습니다.  

이 방식은 튜토리얼 환경에서는 편리하지만, 데이터가 메모리에만 저장되므로 영구적이지 않습니다. 실제 프로덕션 환경에서는 `SqliteSaver` 또는 `PostgresSaver`를 사용하여 데이터베이스(DB)와 연결하는 것이 일반적입니다.

---

In [ ]:
# 상태(State)를 정의한 후, 이를 기반으로 하는 그래프 생성
# LLM에 사용할 도구(tool) 목록을 바인딩함
# chatbot 노드 정의: 현재 상태(state)의 메시지를 LLM에 넘겨 응답 메시지를 반환
def chatbot(state: State):
# chatbot 노드를 그래프에 추가
# 도구 실행 노드 정의 및 추가
# 가장 최근 AI 메시지에 포함된 도구 호출 요청(tool_calls)을 실행하고 결과 메시지 반환
# chatbot → tools 또는 chatbot → END 로 분기되는 조건부 엣지 추가
# route_tools 함수는 마지막 메시지에 도구 호출이 있는지를 확인하여 흐름 결정
# 도구 실행이 끝난 뒤에는 다시 chatbot 으로 돌아가 대화를 계속함
# 그래프의 시작 지점은 chatbot 노드
# 최종적으로 모든 구성 요소를 포함해 그래프를 컴파일함
# memory는 대화 상태를 저장하는 체크포인트 기능 (예: thread_id로 상태 추적)

이제 챗봇과 상호작용할 수 있습니다. 먼저, 이 대화를 식별할 수 있는 **`thread`(스레드)** 를 선택합니다.

In [ ]:
# config는 stream() 또는 invoke()의 두 번째 인자

이제 후속 질문(follow-up question)을 해봅시다. 챗봇이 사용자의 이름을 기억하는지 확인해 봅니다.

In [ ]:
# `thread_id`를 "1" 대신 "2"로 변경합니다.

---
지금까지 두 개의 서로 다른 thread에서 여러 개의 체크포인트를 생성했습니다. 체크포인트에는 어떤 정보가 저장되는 정보는 다음과 같습니다. 

- 현재 상태 값(Current State Values) 
- 해당 상태와 연결된 `config` 정보  
- 다음으로 처리할 노드(Next Node to Process) 

In [ ]:
# 현재 상태 값

In [ ]:
# 해당 상태와 관련된 config 정보

In [ ]:
# 다음으로 처리할 node

### 프롬프트 템플릿(Prompt Templates)을 사용하여 LLM 호출 최적화

**프롬프트 템플릿** 은 **원시 사용자 입력(raw user input)** 을 LLM이 처리할 수 있는 형식으로 변환하는 데 도움을 줍니다.   

1. 먼저, **시스템 메시지(system message)** 를 추가하여 **사용자 정의 지침(custom instructions)** 을 포함시킵니다. (여전히 메시지를 입력으로 사용)  
2. 다음으로, 메시지 외에 **더 많은 입력 정보** 를 추가합니다.  

#### **시스템 메시지(System Message) 추가하기**

시스템 메시지를 추가하기 위해 **`ChatPromptTemplate`** 을 생성합니다. 여기서는 메시지 전달을 위해 **`MessagesPlaceholder`** 를 사용하겠습니다.  

이렇게 하면 LLM에 전달되는 입력이 더 구조화되고, 챗봇의 동작을 더 정교하게 제어할 수 있습니다.  

In [ ]:
# LLM이 사용자 입력을 더 잘 처리할 수 있도록 프롬프트 템플릿을 설정합니다.
        # LLM의 동작 방식을 정의하는 지침
        # Messages Placeholder - 이전 대화 메시지들을 전달합

이제 이 템플릿을 통합하여 애플리케이션을 업데이트할 수 있습니다.

In [ ]:
# 상태 스키마(State)를 정의하여 그래프 생성기 초기화
# LLM에 사용할 도구(tool)들을 바인딩하여, LLM이 도구 호출 기능을 사용할 수 있도록 설정
# 모델 호출 함수 정의
def chatbot(state: State):
    # 현재 상태(state)에 포함된 메시지를 기반으로 프롬프트 생성
    # 생성된 프롬프트를 LLM에 전달하여 응답을 생성
    # LLM 응답을 딕셔너리 형태로 반환 (다음 상태로 전달됨)
# chatbot 노드를 그래프에 추가
# 툴 실행 노드 정의 및 추가
# chatbot 노드에서의 조건부 흐름 정의
# route_tools 함수는 마지막 메시지에 도구 호출이 포함되어 있는지 판단
# 도구 호출이 있으면 "tools" 노드로, 없으면 END로 이동
# MemorySaver를 사용하여 대화 상태를 저장할 메모리 객체 생성
# 도구 실행 후 다시 chatbot 으로 흐름을 연결 (반복 가능)
# 그래프의 시작 지점(START)에서 chatbot 노드로 연결
# 모든 구성 요소를 종합하여 그래프를 컴파일 (메모리를 체크포인터로 사용)
# 컴파일된 그래프 객체를 확인

In [ ]:
# 'configurable' 키를 사용하여 추가적인 설정 값을 전달합니다.
# 여기서는 'thread_id'를 사용하여 특정 대화 스레드를 식별합니다.
# 메시지 목록에 사용자 메시지 추가
# 애플리케이션 호출
# 메시지 상태(State), 설정(config) 전달

In [ ]:
# 'configurable' 키를 사용하여 추가적인 설정 값을 전달합니다.
# 여기서는 'thread_id'를 사용하여 특정 대화 스레드를 식별합니다.
# 메시지 목록에 사용자 메시지 추가
# 애플리케이션 호출
# 메시지 상태(State), 설정(config) 전달

In [ ]:
# 'configurable' 키를 사용하여 추가적인 설정 값을 전달합니다.
# 여기서는 'thread_id'를 사용하여 특정 대화 스레드를 식별합니다.
# 그래프를 메모리 체크포인트와 함께 컴파일합니다.

------------
이제는 chatbot이 외부 정보를 접근할 수도 있고 대화 내용을 기억할 수도 있습니다.